In [5]:
from flask import Flask, request
import os
import sys
import shutil
import urllib.parse
import subprocess
app = Flask(__name__)
#可执行文件绝对地址
compiler_path=sys.path[0]+"/build/SYSY-compiler"
rev_root=sys.path[0]+"/rev"
compiler_args=[compiler_path]
return_src={}

@app.route('/receive_string', methods=['POST'])
def receive_string():
    data = request.get_data().decode('utf-8')
    if not data:
        return "No data received.", 400
    parsed_data = urllib.parse.parse_qs(data)
    #shutil.rmtree(parsed_data)
    file_path=rev_root+"/source_code.sy"    
    os.makedirs(rev_root,exist_ok=True)
    for key, values in parsed_data.items():
        #处理输入的代码+命令
        if(key=="source_code"):
            with open(file_path, 'w') as f:
                for value in values:
                    f.write(value+'\n')
            compiler_args.append(file_path)
        if(key=="params"):
            compiler_args.extend(values)
    #开始进行处理
    result_1=subprocess.run(compiler_args,capture_output=True,text=True)
    with open("/mnt/e/China-GodBolt-forEdu/rev/source.ll", 'w') as f:
        f.write(result_1.stdout)
    with open("/mnt/e/China-GodBolt-forEdu/rev/source.ll", 'r') as f:
        return_src["source_code"] = f.read()
    #对每个pipline都进行一次处理
    result_2=subprocess.run([compiler_path,file_path,'--mem2reg'],capture_output=True,text=True)
    with open("/mnt/e/China-GodBolt-forEdu/rev/mem2reg.ll", 'w') as f:
        f.write(result_2.stdout)
    with open("/mnt/e/China-GodBolt-forEdu/rev/mem2reg.ll", 'r') as f:
        return_src["opt_pipline_mem2reg"]=f.read()
    result_3=subprocess.run([compiler_path,file_path,'--dce'],capture_output=True,text=True)
    with open("/mnt/e/China-GodBolt-forEdu/rev/dce.ll", 'w') as f:
        f.write(result_3.stdout)
    with open("/mnt/e/China-GodBolt-forEdu/rev/dce.ll", 'r') as f:
        return_src["opt_pipline_dce"]=f.read()    
    
    for folder_name in os.listdir(rev_root):
        folder_path = os.path.join(rev_root, folder_name)
        if os.path.isdir(folder_path):
            src=[]
            for file_name in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file_name)
                if os.path.isfile(file_path):
                    with open(file_path, 'r') as f:
                        src.append(f.read())
            return_src[folder_name] = src
        #folder_contents[folder_name] = file_contents
    return return_src
    
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [14/Apr/2024 15:24:19] "POST /receive_string HTTP/1.1" 200 -
